# Projeto Final: Análise Preditiva de Desfechos em Casos de SRAG

**Disciplina:** Introdução à Ciência de Dados – IFB Campus Brasília  
**Estudantes:**  
- Davi Campos Parente  
- Marques Herminio  

---

## Entrega 3 — Preparação Final dos Dados e Início da Modelagem

Esta etapa tem como objetivo preparar os dados para a modelagem preditiva,
realizando transformações finais, seleção de variáveis e construção do
primeiro pipeline de aprendizado de máquina para predição de óbito em casos
de Síndrome Respiratória Aguda Grave (SRAG).


In [16]:
# Biblioteca para manipulação e análise de dados
import pandas as pd
import numpy as np

# Bibliotecas para visualização (podem ser usadas futuramente)
import matplotlib.pyplot as plt

# Ferramentas do scikit-learn para modelagem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


In [17]:
# Instala a biblioteca gdown, usada para baixar arquivos do Google Drive
!pip install -U gdown

# Importa bibliotecas auxiliares
import gdown
import os


In [18]:
os.makedirs('Dados', exist_ok=True)

# Dicionário com os arquivos e seus respectivos IDs públicos do Google Drive
arquivos_drive = {
    'INFLUD19.parquet': '1AZdnTdJrlWurVdeilVVv7svGbfa6_uKa',
    'INFLUD20.parquet': '1TbRazVvlZYkcRZ6E7JPRHAP-_nhs2vnv',
    'INFLUD21.parquet': '1_mc5U3dWG4v2Byqug--QdDZSFc9OVRag',
    'INFLUD22.parquet': '1JTFxGo75-06eDHijLdB-W8krzaMaFSQc',
    'INFLUD23.parquet': '1K5dhMIn81IClZaGJ8NdzpNHXahUo_Z5w',
    'INFLUD24.parquet': '10rcV5cPAJMsXJlax6fOSLx2VIEnMMg-U'
}

for nome, file_id in arquivos_drive.items():
    url = f'https://drive.google.com/uc?id={file_id}'
    output = f'Dados/{nome}'
    if not os.path.exists(output):
        gdown.download(url, output, quiet=False)


In [19]:
# Lista que armazenará os DataFrames individuais
dfs = []

# Leitura de cada arquivo parquet
for arquivo in os.listdir('Dados'):
    if arquivo.endswith('.parquet'):
        df_temp = pd.read_parquet(f'Dados/{arquivo}')

        # Amostragem de 20% para reduzir uso de memória
        df_temp = df_temp.sample(frac=0.2, random_state=42)
        dfs.append(df_temp)

# Concatenação de todos os arquivos em um único DataFrame
df = pd.concat(dfs, ignore_index=True)

# Verificação do tamanho final da base
df.shape


(821913, 194)

In [20]:
# Seleção das variáveis relevantes para o subproblema de predição de óbito
colunas_projeto = [
    'EVOLUCAO', 'CS_SEXO', 'CS_RACA',
    'FEBRE', 'TOSSE', 'DISPNEIA', 'SATURACAO',
    'CARDIOPATI', 'DIABETES', 'OBESIDADE',
    'ASMA', 'IMUNODEPRE', 'UTI', 'SUPORT_VEN',
    'VACINA_COV'
]

# Filtragem do DataFrame
df = df[colunas_projeto]

df.head()


,EVOLUCAO,CS_SEXO,CS_RACA,FEBRE,TOSSE,DISPNEIA,SATURACAO,CARDIOPATI,DIABETES,OBESIDADE,ASMA,IMUNODEPRE,UTI,SUPORT_VEN,VACINA_COV
0,1,F,9,1,1,1,1,None,None,None,None,1,1,1,None
1,1,M,5,1,1,1,1,None,None,None,None,None,2,3,None
2,9,F,9,2,1,1,1,None,None,None,None,None,1,2,None
3,1,F,1,1,1,1,2,None,None,None,None,None,2,2,None
4,None,M,4,1,None,1,None,None,None,None,None,None,2,2,None


In [21]:
# Conversão da variável alvo para formato numérico
df['EVOLUCAO'] = pd.to_numeric(df['EVOLUCAO'], errors='coerce')

# Manter apenas os casos de Cura (1) e Óbito (2)
df = df[df['EVOLUCAO'].isin([1, 2])]

# Transformação em classificação binária:
# 0 = Cura | 1 = Óbito
df['EVOLUCAO'] = df['EVOLUCAO'].map({1: 0, 2: 1})

# Verificação da distribuição da variável alvo
df['EVOLUCAO'].value_counts()


,count
EVOLUCAO,
0,550065
1,179705


In [22]:
# Conversão das variáveis categóricas em variáveis numéricas
# utilizando one-hot encoding
df = pd.get_dummies(df, drop_first=True)

# Dimensão da base após codificação
df.shape


(729770, 33)

In [23]:
# Remoção de registros com valores ausentes
# Opção adotada para simplificar o pipeline inicial
df = df.dropna()

df.shape


(729770, 33)

In [24]:
# X: variáveis explicativas (features)
X = df.drop('EVOLUCAO', axis=1)

# y: variável alvo
y = df['EVOLUCAO']


In [25]:
# Divisão da base em 70% treino e 30% teste
# A estratificação mantém a proporção entre classes
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)


In [26]:
# Padronização das variáveis numéricas
# Necessária para modelos como a Regressão Logística
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [27]:
# Criação e treinamento do modelo de Regressão Logística
# Escolhido por sua interpretabilidade e adequação à classificação binária
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=1000)

In [28]:
# Geração das previsões no conjunto de teste
y_pred = modelo.predict(X_test_scaled)

# Relatório de métricas de classificação
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.82      0.92      0.86    165020
           1       0.60      0.37      0.46     53911

    accuracy                           0.78    218931
   macro avg       0.71      0.64      0.66    218931
weighted avg       0.76      0.78      0.76    218931



In [29]:
# Matriz de confusão
confusion_matrix(y_test, y_pred)


array([[151565,  13455],
       [ 33950,  19961]])